### Extrahera ut personer på Norra Begravningsplatsen från kulturgravar

* https://github.com/salgo60/Gravstensinventeringen-Wikidata/issues/39
* https://www.kulturgravar.se

TODO:
* fixa bättre match med Wikidata
* scanna alla gravar hos kulturgravar
* ge alla kulturgravar ett unikt id kan noig vara dess sida
* stämma av vilka Wikidataobjekt vi redan har koppling till Kulturgravar

In [1]:
import sys
!{sys.executable} -m pip install wikibaseintegrator

In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def find_subpages(base_url):
    try:
        # Fetch the webpage content
        response = requests.get(base_url)
        response.raise_for_status()  # Raise an error for bad HTTP status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {base_url}: {e}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract all anchor tags
    subpages = []
    for link in soup.find_all("a", href=True):
        # Create full URL for relative links
        full_url = urljoin(base_url, link["href"])
        if full_url.startswith(base_url):  # Ensure it's a subpage
            subpages.append(full_url)

    # Remove duplicates by converting to a set
    subpages = list(set(subpages))
    return subpages

# Base URL
base_url = "https://www.kulturgravar.se"

# Find and print subpages

subpagesCounty = find_subpages(base_url)
print(f"Found {len(subpagesCounty)} subpages County:")
for subpageCounty in subpagesCounty:
    print("subpageCounty: ",subpageCounty)
    subpages_county=find_subpages(subpageCounty)
    for sub in subpages_county:
        subpage2=find_subpages(sub)
        print("subpages: ",subpage2,subpage)
        for sub2 in subpage2:
            subpage3=find_subpages(sub2)
            print("subpages: ",subpage3,subpage2, subpage2)
        


Found 16 subpages County:
subpageCounty:  https://www.kulturgravar.se/karta_vastragotalandslan.html
subpageCounty:  https://www.kulturgravar.se/karta_sodermanlandslan.html
subpageCounty:  https://www.kulturgravar.se/index_om.html
subpageCounty:  https://www.kulturgravar.se/karta_uppsalalan.html
subpageCounty:  https://www.kulturgravar.se/karta_hallandslan.html
subpages:  ['https://www.kulturgravar.se/karta_hallandslan.html'] https://www.kulturgravar.se/karta_hallandslan.html
subpages:  ['https://www.kulturgravar.se/karta_hallandslan.html'] ['https://www.kulturgravar.se/karta_hallandslan.html'] ['https://www.kulturgravar.se/karta_hallandslan.html']
subpageCounty:  https://www.kulturgravar.se/karta_ostergotlandslan.html
subpageCounty:  https://www.kulturgravar.se/karta_varmlandslan.html
subpageCounty:  https://www.kulturgravar.se/karta_blekingelan.html
subpageCounty:  https://www.kulturgravar.se/index_kontakt.html
subpageCounty:  https://www.kulturgravar.se/karta_orebrolan.html
subpageCo

In [ ]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag 
# URL of the webpage to scrape
urllist = {
           "https://www.kulturgravar.se/6_14_500_SV.html",
           "https://www.kulturgravar.se/6_14_500_SO.html"} #NorraBegravningsplatsen

#urllist = {           "https://www.kulturgravar.se/6_14_263_karta.html",
#           "https://www.kulturgravar.se/6_14_500_SV.html",
#           "https://www.kulturgravar.se/6_14_500_SO.html"}

url="https://www.kulturgravar.se" 

In [ ]:

all_points = []

# Send a GET reaquest to fetch the page content
for url in urllist:     
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        print("Page fetched successfully!")
    else:
        print(f"Failed to fetch the page. Status code: {response.status_code}")
        exit()
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Locate the specific elements containing the list of people
    # Based on the structure of the page source, adjust the selector
    people = [] 
    
    # Find all the required elements
    points = soup.find_all('div', class_='point')
    if points:  # Ensure current_results is not None
        all_points.extend(points)


# Extract details
results = []
#print(all_points)
for points in  all_points:
    #print(len(points), points)

    for point in points:
        # Ensure the current element is a bs4.element.Tag
        if isinstance(point, Tag):
            try:
                # Extract the name and dates (from abbr title)
                label_title = point.find('abbr')['title']
            
                # Append the result as a tuple
                results.append(label_title)
            except:
                print("\tError title")
                print(point)

# Print the extracted data
for person in results:
    print(f"Name and Dates: {person}")


In [ ]:
from wikibaseintegrator.wbi_helpers import execute_sparql_query
from wikibaseintegrator import WikibaseIntegrator 
from wikibaseintegrator.wbi_config import config as wbi_config

In [ ]:
wbi_config['USER_AGENT'] = 'WikibaseIntegrator in PAWS by salgo60'
wbi = WikibaseIntegrator()

def get_qnumber(name):
        
    # Query to search for the person by label
    query = f"""
    SELECT ?person ?personLabel WHERE {{
        ?person ?label "{name}"@sv. # Swedish label
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    LIMIT 1
    """
    
    # Execute the query
    try:
        results = execute_sparql_query(query)
        bindings = results["results"]["bindings"]
        print(f"Found {len(bindings)} results")

        if bindings:
            return bindings[0]['person']['value'].split('/')[-1]  # Extract Qnumber
        else:
            return None
    except Exception as e:
        print(f"Error searching for {name}: {e}")
        return None

# Your list of results


for person in results:
    name_and_dates = person
    name = name_and_dates.split(' (')[0]  # Extract name only
    qnumber = get_qnumber(name)
    
    print(f"File Link: {person[0]}")
    print(f"Grave Title: {person[1]}")
    print(f"Name and Dates: {name_and_dates}")
    print(f"Wikidata Qnumber: {qnumber if qnumber else 'Not found'}")
    print("-" * 40)
